In [1]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

/Users/khoa/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
data = pd.read_csv('text_dataset.csv') #Dataset generated from the wrangler file
train, test = train_test_split(data, random_state=745) #Split the dataset into training and test

In [4]:
train['label'].value_counts()

suicide          87069
non-suicide      87060
anxious           7809
normal            7459
stressed          6436
lonely            6384
not depressed     5958
depressed         1734
Name: label, dtype: int64

In [5]:
test.drop('label', axis=1, inplace=True)
test.sample(5)

,text
208853,"['hate', 'indian', 'uncl', 'recent', 'parent',..."
210693,"['number', 'game', '!', 'saw', 'lot', 'recent'..."
169378,"['rock', 'bottomi', 'ca', ""n't"", 'go', 'anymor..."
137936,"['everyon', 'appreci', 'post', 'deserv', 'appr..."
271841,"['listen', 'say', 'know', 'everyon', 'say', 'q..."


In [6]:
def changing_label_suicide(row):
    if row['label'] == 'suicide':
        return 1
    return 0

def changing_label_lonely(row):
    if row['label'] == 'lonely':
        return 1
    return 0

def changing_label_stressed(row):
    if row['label'] == 'stressed':
        return 1
    return 0

def changing_label_normal(row):
    if row['label'] == 'normal':
        return 1
    return 0

def changing_label_non_suicide(row):
    if row['label'] == 'non-suicide':
        return 1
    return 0

def changing_label_not_depressed(row):
    if row['label'] == 'not depressed':
        return 1
    return 0

def changing_label_depressed(row):
    if row['label'] == 'depressed':
        return 1
    return 0

def changing_label_anxious(row):
    if row['label'] == 'anxious':
        return 1
    return 0

In [7]:
train['suicide'] = 0
train['lonely'] = 0
train['stressed'] = 0
train['normal'] = 0
train['non-suicide'] = 0
train['not depressed'] = 0
train['depressed'] = 0
train['anxious'] = 0

train['suicide'] = train.apply(lambda row: changing_label_suicide(row), axis=1)
train['non-suicide'] = train.apply(lambda row: changing_label_non_suicide(row), axis=1)

train['not depressed'] = train.apply(lambda row: changing_label_not_depressed(row), axis=1)
train['depressed'] = train.apply(lambda row: changing_label_depressed(row), axis=1)

train['lonely'] = train.apply(lambda row: changing_label_lonely(row), axis=1)
train['stressed'] = train.apply(lambda row: changing_label_stressed(row), axis=1)
train['normal'] = train.apply(lambda row: changing_label_normal(row), axis=1)
train['anxious'] = train.apply(lambda row: changing_label_anxious(row), axis=1)


train.sample(5)

,text,label,suicide,lonely,stressed,normal,non-suicide,not depressed,depressed,anxious
194526,"['dont', 'deserv', 'lifeit', ""'s"", 'hard', 'th...",suicide,1,0,0,0,0,0,0,0
88201,"['realli', 'big', 'oof', 'start', 'watch', '``...",non-suicide,0,0,0,0,1,0,0,0
182899,"['hey', 'girl', ',', 'tree', '?', 'cu', 'hear'...",non-suicide,0,0,0,0,1,0,0,0
93056,"['life', 'worth', 'live', 'anymor', '?', 'hey'...",suicide,1,0,0,0,0,0,0,0
242015,"['anyon', 'wan', 'na', 'chat', 'someth', 'idk'...",non-suicide,0,0,0,0,1,0,0,0


In [8]:
layer_1 = ['suicide', 'non-suicide']
layer_2 = ['depressed', 'not depressed']
layer_3 = ['normal', 'lonely', 'stressed', 'anxious']


In [9]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Some additional clean up. Idk if this is needed. 

n = train.shape[0]
vec = TfidfVectorizer(analyzer = "word", max_features=10000)
X_dtm = vec.fit_transform(train['text'])
test_X_dtm = vec.transform(test['text'])


In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
suicide_pred_model = DecisionTreeClassifier()
layer_1 = ['suicide', 'non-suicide']

for label in layer_1:
    print('... Processing {}'.format(label))
    y = train[label]
    suicide_pred_model.fit(X_dtm, y)
    y_pred_X = suicide_pred_model.predict(X_dtm)
    print('Training accuracy is {}'.format(accuracy_score(y, y_pred_X)))



In [11]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
depression_pred_model = LinearSVC()
layer_2 = ['depressed', 'not depressed']

for label in layer_2:
    print('... Processing {}'.format(label))
    y = train[label]
    depression_pred_model.fit(X_dtm, y)
    y_pred_X = depression_pred_model.predict(X_dtm)
    print('Training accuracy is {}'.format(accuracy_score(y, y_pred_X)))


fit suicide
fit non-suicide


In [12]:
#Doesn't work yet.
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score
other_cons_pred_model = BernoulliNB()
layer_3 = ['normal', 'lonely', 'stressed', 'anxious']
pred = []


for i,j in enumerate(layer_3):
    print('... Processing {}'.format(j))
    y = train[j]
    pred.append(other_cons_pred_model.fit(X_dtm, y))
    y_pred_X = other_cons_pred_model.predict(X_dtm)
    print('Training accuracy is {}'.format(accuracy_score(y, y_pred_X)))



fit depressed
fit not depressed
fit normal
fit lonely
fit stressed
fit anxious


In [ ]:
from joblib import dump, load
dump(suicide_pred_model, 'suicide_prediction.joblib') 
dump(depression_pred_model, 'depression_prediction.joblib') 

In [ ]:
txt = "I am happy"
vtxt = vec.transform([txt])
predsx = np.zeros((len(layer_1)))
for i, j in enumerate(layer_1):
    predsx = suicide_pred_model.predict_proba(vtxt)[0]
print(predsx)

def get_lab_proba(predsx,threshold=0.1):
    out = predsx.tolist()
    max = 0
    lab = "not toxic"
    for i in range(len(out)):
        if out[i] > max and out[i]>threshold:
            max = out[i]
            lab = layer_1[i]
    print(lab,max)
get_lab_proba(predsx)

#Input prediction for suicide

In [ ]:
txt = "I am sad"
vtxt = vec.transform([txt])
predsx = np.zeros((len(layer_2)))
for i, j in enumerate(layer_2):
    predsx = suicide_pred_model.predict_proba(vtxt)[0]
print(predsx)

def get_lab_proba(predsx,threshold=0.1):
    out = predsx.tolist()
    max = 0
    lab = "not toxic"
    for i in range(len(out)):
        if out[i] > max and out[i]>threshold:
            max = out[i]
            lab = layer_2[i]
    print(lab,max)
get_lab_proba(predsx)

#text prediction for depression